In [ ]:
#@title <font size=2pt>Imports</font>
from google.colab import files
import pandas as pd
import plotly.graph_objects as go
from plotly.graph_objs import Line
from google.colab import data_table

[Solving the protein folding problem in hydrophobic-polar model using deep reinforcement learning](https://link.springer.com/article/10.1007/s42452-020-2012-0)

Protein Folding Problem: Process of predicting the structure of a protein given its amino acids sequence.

This paper puts forth a method for solving the bi-dimensional protein folding problem when represented with a hydrophobic-polar(HP) model. The HP model is a lattice-based model that represents each amino acid as either Hydrophobic, denoted by H, or Hydrophillic (polar), denoted by P.

Each protein is initially represented as a sequence of amino acids (H or P). This sequence is then conformed into a 2 dimensional lattice, such that consecutive amino acid in the sequence are neighbors in the lattice. The goal is to find the lattice configuration that gives you the lowest possible free energy, which will give you the stable state of the protein.

DRL Approach:


Preprocess: Represent the 2 dimensional lattice as a matrix where $-1$ represents the Hydrophobic amino acids, $1$ represents the Hydrophilic (polar) amino acids and $0$ represents an empty space.

\\

Markov Decision Process (MDP):
* Environment: Lattice-Matrix

* State: 3 pieces of information
    1. The type of amino acid $\{-1, 1\}$
    2. Position of the amino acid in the matrix $(row, col)$
    3. The order of the amino acid in the inital sequence $\{1, 2, \cdots, n\}$

* Action: $\{\text{Up}, \text{Down}, \text{Left}, \text{Right}\}$

* Reward: Amino acids have a tendency to form a hydrophobic core in their stable state, therefore the agent receives a $-1$ reward for each pair of non-consecutive amino acids that end up as neighbors in the lattice. We are minimizing the free energy function, thus the negative reward.


\\

Actor - Critic Algorithm:

* Actor (Policy): Outputs preferences for each action that are later converted to probabilities using a softmax

* Critic (Q-function): Outputs the Q-value of each action, given a state.

The actor updates the policy distribution in the direction suggested by the critic.

Both networks use 3 hidden LSTM layers, with each layer followed by a dropout layer. The critic used a dueling-network architecture and experience replay mechanism.



In [ ]:
#@title Benchmark Sequnces

df0 = pd.read_csv('Sequences.csv', index_col=False)
df0.set_index('Sequence number', inplace=True)
data_table.DataTable(df0)

,Length,Protein sequence
Sequence number,,
1,20,(HP)2PH2PHP2HPH2P2HPH
2,24,H2P2(HP2)6H2
3,25,P2HP2H2P4H2P4H2P4H2
4,36,P3H2P2H2P5H7P2H2P4H2P2HP2
5,48,P2HP2H2P2H2P5H10P6H2P2H2P2HP2H5
6,50,H2(PH)3PH4PHP3HP3HP4HP3HP3HPH4(PH)3PH2
7,60,P2H3PH8P3H10PHP3H12P4H6PH2PHP
8,64,H12(PH)2(P2H2)2P2H(P2H2)2P2HPHPH12
9,85,H4P4H12P6(H12P3)3HP2(H2P2)2HPH


In [ ]:
#@title Table 2: Free Energy Comparison
df1 = pd.read_csv('FreeEnergyComparison.csv', index_col=False)
df1.set_index('Sequence number', inplace=True)
data_table.DataTable(df1)

,ACO,FoldingZero,GA,EMC,ENLS,PERM,DRL
Sequence number,,,,,,,
1,− 9,− 9,− 9,− 9,− 9,− 9,− 9
2,− 9,− 8,− 9,− 9,− 9,− 9,− 9
3,− 8,− 7,− 8,− 8,− 8,− 8,− 8
4,− 14,− 13,− 14,− 14,− 14,− 14,− 14
5,− 23,− 18,− 23,− 23,− 23,− 23,− 23
6,− 21,− 18,− 21,− 21,− 21,− 21,− 21
7,− 36,− 32,− 34,− 35,− 36,− 36,− 36
8,− 42,NaN,− 37,− 39,− 39,− 38,− 42
9,− 53,− 48,NaN,− 52,NaN,− 53,− 53


In [ ]:
#@title Table 3: Performance Comparison

df2 = pd.read_csv('PerformanceComparison.csv')
df2.set_index('Sequence number', inplace=True)
data_table.DataTable(df2)

,ACO,GA,EMC,PERM,DRL
Sequence number,,,,,
1,Less than a second,"30,492",9374,Less than a second,Less than a second
2,Less than a second,"30,491",6929,Less than a second,Less than a second
3,Less than a second,"20,400",7202,6 s,Less than a second
4,4 s,"501,339","12,447",Less than a second,Less than a second
5,1 min,"126,547","165,791",3 min,25 s
6,15 s,"592,887","74,613",3 s,Less than a second
7,20 min,"208,781","203,729",7 s,Less than a second
8,1.5 h,"187,393","564,809",78 h,48 min
9,A day,NaN,"44,029",64 s,16 s


Comparisons References:

[ACO](https://link.springer.com/article/10.1186/1471-2105-6-30)

[Folding Zero](https://deepmind.com/blog/article/AlphaFold-Using-AI-for-scientific-discovery)

[GA](https://www.sciencedirect.com/science/article/abs/pii/S0022283683712581?via%3Dihub)

[EMC](https://aip.scitation.org/doi/10.1063/1.1387478)

[ENLS](https://aip.scitation.org/doi/abs/10.1063/1.2357950)

[PERM](https://onlinelibrary.wiley.com/doi/abs/10.1002/(SICI)1097-0134(19980701)32:1%3C52::AID-PROT7%3E3.0.CO;2-G)

[DRL](https://link.springer.com/article/10.1007/s42452-020-2012-0#Sec11) (this paper)



# Critiques

* Instead of a binary option (H or P), we can have relative rankings for the hydrophobicity of different amino acids and incorporate this into our reward function
  * For instance, the more hydrophobic a residue is, the more reward will be given if this molecule is placed in the middle of a molecule (hydrophobic core).
  * Doolittle Hydrophobicity scale could be used to weight residues
* This would necessitate the use of the actual amino acid sequence as opposed to only H and P
* Now, using the full amino acid sequence, one could begin to incorporate other forces
* We can use A2C, assuming they used a Q-valued actor-critic algorithm